In [3]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [4]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

In [29]:
import collect_stats as cs

In [25]:
filenames = [
    'HOXC4_R158L_R1_8mers.txt',
    'HOXD13_REF_R1_8mers.txt',
    'GFI1_REF_R1_8mers.txt',
    'HOXC4_R158L_R2_8mers.txt',
    'PAX4_REF_R1_8mers.txt',
    'OVOL2_D228E_R1_8mers.txt',
    'POU3F4_V322M_R1_8mers.txt',
    'KLF11_R402Q_R1_8mers.txt',
    'HOXB7_R189C_R1_8mers.txt',
    'CRX_R90W_R1_8mers.txt',
]

In [ ]:
mves_condense_compare_w_old = cs.get_data_mves('/cluster/sj1/bb_opt', 'e')

In [5]:
mves = cs.get_data_mves('/cluster/sj1/bb_opt', 'n')

reading /cluster/sj1/bb_opt/output_n1/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n1/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n2/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n2/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n2/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n3/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n3/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n3/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n4/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n4/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n4/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n5/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n5/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n5/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n6/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n6/HOXD13_REF_R1_8mers.txt
reading /c

In [129]:
mves_condense = cs.get_data_mves('/cluster/sj1/bb_opt', 'f')

reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10


In [148]:
mves = cs.get_data_mves('/cluster/sj1/bb_opt', 'mves')

reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 9
reading sample 10


In [141]:
pdts = cs.get_data_mves('/cluster/sj1/bb_opt', 'pdts_dna_n')

reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9


In [142]:
pdts_mix = cs.get_data_mves('/cluster/sj1/bb_opt', 'pdts_dna_cond_n')

reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10


In [130]:
mves_ei_mix = cs.get_data_mves('/cluster/sj1/bb_opt', 'ei_mves_mix')

reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10


In [55]:
ucb = cs.get_data_ucb('/cluster/sj1/bb_opt', 'n')

reading 0
read 10 samples
reading 1
read 10 samples
reading 2
read 10 samples
reading 3
read 10 samples
reading 4
read 10 samples


In [150]:
batch_idx = 1
ack_iter_idx = 19
stat_idx = -4

In [152]:
for f in filenames:
    print(f)
    try:
        arr = mves_ei_mix[f][batch_idx]
        arr_top = arr[:, ack_iter_idx, stat_idx]
        print('mves_ei_mix:', arr.shape[0], np.mean(arr_top), np.std(arr_top), np.median(arr[:, ack_iter_idx, -5]))
    except Exception as e:
        pass

        
    try:
        arr = pdts_mix[f][batch_idx]
        arr_top = arr[:, ack_iter_idx, stat_idx]
        print('pdts*2:', arr.shape[0], np.mean(arr_top), np.std(arr_top), np.median(arr[:, ack_iter_idx, -5]))
    except Exception as e:
        pass
    
    try:
        arr = pdts[f][batch_idx]
        arr_top = arr[:, ack_iter_idx, stat_idx]
        print('pdts:', arr.shape[0], np.mean(arr_top), np.std(arr_top), np.median(arr[:, ack_iter_idx, -5]))
    except Exception as e:
        pass
    
        
    try:
        arr = mves[f][batch_idx]
        arr_top = arr[:, ack_iter_idx, stat_idx]
        print('mves:', arr.shape[0], np.mean(arr_top), np.std(arr_top), np.median(arr[:, ack_iter_idx, -5]))
    except Exception as e:
        pass  
    
    try:
        arr = mves_condense[f][batch_idx][:, ack_iter_idx, stat_idx]
        print('ei*2:', mves_condense[f][batch_idx].shape[0], np.mean(arr), np.std(arr), np.median(arr))
    except Exception as e:
        pass
    
    ucb_coeffs = [0.0, 0.5, 1.0, 2.0, 3.0]
    for i in range(5):
        try:
            if len(ucb[i][f][batch_idx]) != 0:
                arr = ucb[i][f][batch_idx][:, ack_iter_idx, stat_idx]
                print('ucb:', ucb_coeffs[i], ucb[i][f][batch_idx].shape[0], np.mean(arr), np.std(arr), np.median(arr))
        except Exception as e:
            pass

HOXC4_R158L_R1_8mers.txt
pdts*2: 10 0.12042682 0.017738037 67918.92
pdts: 9 0.05182927 0.00964109 66866.02
ei*2: 10 0.15304878 0.015771361 0.1570122
ucb: 0.0 10 0.104573175 0.014688036 0.10060975
ucb: 0.5 10 0.10579269 0.02177478 0.10670732
ucb: 1.0 10 0.09390243 0.019701824 0.085365854
ucb: 2.0 10 0.1015244 0.02194487 0.09146342
ucb: 3.0 10 0.10121951 0.012702846 0.09908536
HOXD13_REF_R1_8mers.txt
pdts*2: 10 0.11189024 0.015428757 158176.05
pdts: 9 0.056910567 0.016260162 146057.94
ei*2: 10 0.13932927 0.0143033 0.14176829
ucb: 0.0 10 0.09603658 0.013095567 0.092987806
ucb: 0.5 10 0.09908537 0.010316418 0.09756097
ucb: 1.0 10 0.08810976 0.015750721 0.09146342
ucb: 2.0 10 0.08384146 0.019294228 0.07926829
ucb: 3.0 10 0.08871951 0.008563764 0.08841463
GFI1_REF_R1_8mers.txt
pdts*2: 10 0.024085365 0.027824193 209234.6
pdts: 9 0.018292684 0.013093597 174996.73
ei*2: 10 0.069817066 0.043362405 0.074695125
ucb: 0.0 10 0.02378049 0.025718411 0.013719512
ucb: 0.5 10 0.03628049 0.0262071 0.04268

In [138]:
mves_condense_chemvae = cs.get_data_mves('/cluster/sj1/bb_opt', 'ei_cond_chemvae')

reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10


In [90]:
ucb_chemvae = cs.get_data_ucb('/cluster/sj1/bb_opt', 'chemvae')

reading 0
read 10 samples
reading 1
read 0 samples
reading 2
read 2 samples
reading 3
read 0 samples
reading 4
read 0 samples


In [134]:
batch_idx = 1
ack_iter_idx = -1
stat_idx = -4
f = 'chemvae'

In [139]:
try:
    data = mves_condense_chemvae[f][batch_idx]
    arr = data[:, ack_iter_idx, stat_idx]
    print('mves_ei*2:', data.shape[0], np.mean(arr), np.std(arr), np.median(arr), np.median(data[:, ack_iter_idx, -5]))
except Exception as e:
    print(e)

ucb_coeffs2 = [0.0, 0.5, 1.0]
for i in range(len(ucb_coeffs2)):
    try:
        if len(ucb_chemvae[i][f][batch_idx]) != 0:
            arr = ucb_chemvae[i][f][batch_idx][:, ack_iter_idx, stat_idx]
            print('ucb:', ucb_coeffs2[i], ucb_chemvae[i][f][batch_idx].shape[0], np.mean(arr), np.std(arr), np.median(arr), np.median(ucb_chemvae[i][f][batch_idx][:, ack_iter_idx, -5]))
    except Exception as e:
        pass

mves_ei*2: 10 0.004851644 0.0027221178 0.003809142 2.8603437
ucb: 0.0 10 0.0031676025 0.0023547788 0.0024057738 2.7301798
ucb: 1.0 2 0.0028067362 0.0012028869 0.0028067362 2.8094695
